<a href="https://colab.research.google.com/github/santhimaddipudi/Python-work/blob/master/Python_polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df=pd.read_csv("/content/sample_data/Iris.csv")
df_filtered = df[df["SepalLengthCm"] > 5].groupby("Species").agg('sum')  # Filter rows where 'SepalLengthCm' is greater than 5
df_filtered.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,,
Iris-setosa,564,116.9,81.7,33.2,6.1
Iris-versicolor,3562,281.9,131.8,202.9,63.3
Iris-virginica,6168,324.5,146.2,273.1,99.6


In [3]:
import polars as pl
pl.__version__

'1.21.0'

In [ ]:
#operations that are supported on materialized dataframes but not on lazy dataframes.
df_ops = set(x for x in dir(pl.DataFrame()) if not x.startswith('_'))
lazy_ops = set(x for x in dir(pl.LazyFrame()) if not x.startswith('_'))
print(sorted(df_ops - lazy_ops))

['apply', 'corr', 'describe', 'drop_in_place', 'equals', 'estimated_size', 'extend', 'find_idx_by_name', 'flags', 'fold', 'frame_equal', 'get_column', 'get_column_index', 'get_columns', 'glimpse', 'hash_rows', 'height', 'hstack', 'insert_at_idx', 'insert_column', 'is_duplicated', 'is_empty', 'is_unique', 'item', 'iter_columns', 'iter_rows', 'iter_slices', 'map_rows', 'max_horizontal', 'mean_horizontal', 'min_horizontal', 'n_chunks', 'n_unique', 'partition_by', 'pivot', 'product', 'rechunk', 'replace', 'replace_at_idx', 'replace_column', 'row', 'rows', 'rows_by_key', 'sample', 'shape', 'shrink_to_fit', 'sum_horizontal', 'to_arrow', 'to_dict', 'to_dicts', 'to_dummies', 'to_init_repr', 'to_numpy', 'to_pandas', 'to_series', 'to_struct', 'transpose', 'unstack', 'upsample', 'vstack', 'write_avro', 'write_csv', 'write_database', 'write_delta', 'write_excel', 'write_ipc', 'write_ipc_stream', 'write_json', 'write_ndjson', 'write_parquet']


In [ ]:
import polars as pl

q = (
    pl.scan_csv("/content/sample_data/Iris.csv")
    .filter(pl.col("SepalLengthCm") > 5)
    .group_by("Species")
    .agg(pl.all().sum())
)

df = q.collect()

df

Species,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
str,i64,f64,f64,f64,f64
"""Iris-virginica…",6168,324.5,146.2,273.1,99.6
"""Iris-setosa""",564,116.9,81.7,33.2,6.1
"""Iris-versicolo…",3562,281.9,131.8,202.9,63.3


In [4]:
#for lazy dataframe it wont show the data
df1=pl.scan_csv("/content/sample_data/Iris.csv").filter(pl.col("SepalLengthCm") > 5)
print(df1)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

FILTER [(col("SepalLengthCm")) > (5.0)] FROM
  Csv SCAN [/content/sample_data/Iris.csv]
  PROJECT */6 COLUMNS


In [7]:
col=pl.col("SepalLengthCm")
col_ops = set(x for x in dir(col) if not x.startswith('_'))
print(col_ops)

{'shuffle', 'truediv', 'forward_fill', 'sample', 'rolling_min_by', 'replace_strict', 'arr', 'slice', 'arg_true', 'cut', 'entropy', 'ewm_mean', 'extend_constant', 'over', 'is_null', 'ewm_var', 'dt', 'map_batches', 'mod', 'reshape', 'shift', 'sin', 'explode', 'floordiv', 'rolling_std', 'log', 'rolling_var_by', 'rolling_sum', 'is_unique', 'rolling_var', 'tanh', 'not_', 'index_of', 'gather', 'fill_null', 'from_json', 'bottom_k', 'where', 'is_not_null', 'le', 'str', 'gather_every', 'std', 'interpolate_by', 'get', 'tan', 'diff', 'abs', 'cast', 'sort', 'top_k_by', 'repeat_by', 'inspect', 'rolling_skew', 'cum_max', 'rolling', 'is_finite', 'is_not_nan', 'len', 'meta', 'nan_max', 'struct', 'rolling_quantile_by', 'mul', 'bitwise_or', 'log1p', 'bitwise_leading_zeros', 'ewm_std', 'has_nulls', 'add', 'value_counts', 'cum_prod', 'floor', 'replace', 'n_unique', 'drop_nans', 'mode', 'rolling_sum_by', 'rolling_max', 'cum_min', 'rle_id', 'kurtosis', 'arctanh', 'cos', 'log10', 'product', 'search_sorted', 

In [ ]:
df=pd.read_csv("/content/sample_data/vehicles.csv")
print(df)

In [16]:
lazy = pl.scan_csv("/content/sample_data/vehicles.csv", null_values=['NA'])
print(lazy)
lazy.explain(optimized=True)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [/content/sample_data/vehicles.csv]
PROJECT */84 COLUMNS


'Csv SCAN [/content/sample_data/vehicles.csv]\nPROJECT */84 COLUMNS'

In [18]:
print(lazy
 .filter((pl.col('year') >= 1990) & (pl.col('year') < 2000))
 .select(['year','make'])
 .explain()
)

simple π 2/2 ["year", "make"]
  Csv SCAN [/content/sample_data/vehicles.csv]
  PROJECT 2/84 COLUMNS
  SELECTION: [([(col("year")) < (2000)]) & ([(col("year")) >= (1990)])]


In [19]:
print(df.dtypes)

barrels08     float64
barrelsA08    float64
charge120     float64
charge240     float64
city08          int64
               ...   
modifiedOn     object
startStop      object
phevCity        int64
phevHwy         int64
phevComb        int64
Length: 84, dtype: object


In [ ]:
print(df.info(verbose=True))

In [29]:
cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08','createdOn']
df_polars = pl.from_pandas(df)
print(df_polars.select(cols))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i64  ┆ str        ┆ str            ┆ f64   ┆   ┆ f64       ┆ i64    ┆ i64       ┆ str            │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ Tue Jan 01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00 EST   │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 2013           │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ Tue Jan 01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆

In [ ]:
print(df_polars.select(pl.col(cols)))

In [31]:
print(df_polars
 .select(pl.col(cols))
 .select(pl.col(pl.Int64))
 .describe())

shape: (9, 4)
┌────────────┬─────────────┬───────────┬───────────┐
│ statistic  ┆ year        ┆ city08    ┆ highway08 │
│ ---        ┆ ---         ┆ ---       ┆ ---       │
│ str        ┆ f64         ┆ f64       ┆ f64       │
╞════════════╪═════════════╪═══════════╪═══════════╡
│ count      ┆ 48581.0     ┆ 48581.0   ┆ 48581.0   │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0       │
│ mean       ┆ 2004.752413 ┆ 20.131615 ┆ 25.970256 │
│ std        ┆ 12.758499   ┆ 13.198585 ┆ 11.439295 │
│ min        ┆ 1984.0      ┆ 6.0       ┆ 9.0       │
│ 25%        ┆ 1993.0      ┆ 15.0      ┆ 20.0      │
│ 50%        ┆ 2006.0      ┆ 18.0      ┆ 24.0      │
│ 75%        ┆ 2016.0      ┆ 21.0      ┆ 28.0      │
│ max        ┆ 2026.0      ┆ 153.0     ┆ 142.0     │
└────────────┴─────────────┴───────────┴───────────┘


In [ ]:
#When the with_columns context runs, Polars will read the data from the year column and cast it to a 16-bit integer
and cast it to a 16-bit integerprint(df_polars
 .select(pl.col(cols))
 .with_columns(pl.col('year').cast(pl.Int16),
               pl.col('cylinders').cast(pl.UInt8),
               pl.col('highway08').cast(pl.UInt8),
               pl.col('city08').cast(pl.UInt8)))

In [36]:
#Polars includes a column expression to shrink numeric columns, the shrink_dtype method
print(df_polars.select(pl.col(cols).shrink_dtype()))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i16  ┆ str        ┆ str            ┆ f32   ┆   ┆ f32       ┆ i16    ┆ i16       ┆ str            │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ Tue Jan 01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00 EST   │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 2013           │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ Tue Jan 01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆